<a href="https://colab.research.google.com/github/nferrucho/NPL/blob/main/curso1/ciclo5/1_agrupamiento_texto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://drive.google.com/uc?export=view&id=1WNLKH10YpQNNk9eeRIyYLwGkxNbNp-Mm" width="100%">

# Agrupamiento de Textos
---

En este notebook veremos una introducción al aprendizaje no supervisado en procesamiento de lenguaje natural con un énfasis en agrupamiento de textos o *text clustering*. Comenzamos importando las librerías necesarias:

In [ ]:
!pip install unidecode

In [ ]:
import re
import spacy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from unidecode import unidecode
from IPython.display import display

## **1. Análisis no Supervisado en Textos**
---

Hasta el momento, hemos visto técnicas de aprendizaje supervisado para casos con textos etiquetados. Sin embargo, una gran cantidad de la información textual existente no contiene etiquetas, por lo que no es posible entrenar un modelo de clasificación. En estos casos, se deben aplicar modelos descriptivos usando técnicas de aprendizaje no supervisado. Esto con el fin de explicar el comportamiento de los datos, ya que podría contener información semántica de gran utilidad.

Este tipo de modelos descriptivos tiene gran aplicación, principalmente en el entendimiento de grandes corpus a través de la identificación de los temas comunes entre varios documentos. Con este tipo de modelos podemos encontrar:

- **Opiniones**: identificar opiniones o tendencias en documentos sobre una temática en particular, por ejemplo, en política, deportes, creencias, entre otras.
- **Temas**: permite encontrar diversos temas dentro de un corpus, por ejemplo, géneros literarios en una biblioteca, tipos de noticias, género de películas o series a partir de su sinopsis, entre otros.
- **Estilos**: permite encontrar tipos de escritura de distintos autores, por ejemplo, escritos formales, lenguaje coloquial, escritos técnicos, entre otros.

Por lo general, el análisis no supervisado de texto es una tarea que actualmente consiste en la aplicación de un modelo no supervisado de *machine learning*; en la siguiente figura podemos ver el proceso general de esta tarea:

<img src="https://drive.google.com/uc?export=view&id=18xoF0c2qREKfH4yYQA0xR8koKFnAkeXH" width="80%">

Lo cual comprende las siguientes etapas:

1. **Preprocesamiento**: consiste en la preparación de los textos para simplificar el entrenamiento de un modelo. Generalmente esta etapa involucra tareas como normalización, tokenization, eliminación de stopwords, y otras tareas tal y como se presentó en la unidad 2.
2. **Extracción de características**: en este caso se extrae una representación numérica o características con las que se pueda modelar, tal y como se presentó en la unidad 3.
3. **Construcción del modelo**: en esta etapa se entrena un modelo no supervisado de _machine learning_ (por ejemplo, clustering, reducción de dimensionalidad, modelos generativos, entre otros) o modelos no supervisados específicos para procesamiento de lenguaje natural (modelos de tópicos).
4. **Interpretación del modelo**: en esta última etapa se suele dar una interpretación a los resultados obtenidos, en especial, validar si los patrones extraídos por los modelos tienen algún significado y aplicabilidad.

## **2. Conjunto de Datos**
---

En este caso usaremos el conjunto de datos [TMDB 5000 Movie Dataset](https://www.kaggle.com/tmdb/tmdb-movie-metadata/download), el cual contiene información sobre películas como su título, presupuesto, resumen, popularidad, géneros, entre otras.

<img src="https://drive.google.com/uc?export=view&id=1D-y9Vu-TxrenTRtd-hmBU-Fi3M15Th7h" width="80%">

Procedemos a cargar el conjunto de datos (simplificado):

In [ ]:
df = pd.read_parquet("https://raw.githubusercontent.com/mindlab-unal/mlds4-datasets/main/u5/tmdb5000.parquet").dropna()
display(df.head())

Se trata de un conjunto de datos que contiene 4800 películas:

In [ ]:
display(df.shape[0])

En este caso usaremos únicamente algunos campos relevantes para el análisis no supervisado de textos, estos son:

- `title`: título de la película.
- `overview`: resumen de la película.
- `genres`: listado de géneros asociados a la película.

Nuestro corpus estará conformado principalmente por el texto en el campo `overview`; vamos a definir una función de preprocesamiento para dejarlo más limpio. Primero definimos un pipeline en blanco de `spacy` (para tokenizar y eliminar stopwords).

In [ ]:
nlp = spacy.blank("en")
display(nlp)

Ahora, definimos la función de preprocesamiento:

In [ ]:
def preprocess(text):
    doc = nlp(text) # creamos un documento de spacy
    no_stops = " ".join(
        token.text
        for token in filter(
            lambda token: not token.is_stop and len(token) > 3 and len(token) < 24,
            doc,
            )
        ) # eliminamos stopwords y palabras por longitud
    norm_text = unidecode(no_stops.lower()) # normalizamos el texto
    no_chars = re.sub(r"[^a-z ]", " ", norm_text) # eliminamos caracteres especiales
    no_spaces = re.sub(r"\s+", " ", no_chars) # eliminamos espacios duplicados
    return no_spaces.strip()

Aplicamos la función sobre todo el corpus:

In [ ]:
df = df.assign(
        norm_text = df.overview.apply(preprocess)
        )
display(df.head())

## **3. Extracción de Características**
---

En este caso utilizaremos una representación de tipo _TF-IDF_. Comenzamos importando el vectorizador:

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

Ahora, entrenamos el vectorizador sobre el corpus preprocesado, extraemos únicamente los términos que aparecen como mínimo en 5 documentos (`min_df`).

In [ ]:
vect = TfidfVectorizer(sublinear_tf=True, min_df=0.05).fit(df.norm_text)
display(vect)

Ahora extraemos la representación:

In [ ]:
features = vect.transform(df.norm_text).toarray()
display(features.shape)

Como podemos ver, el conjunto de datos tiene casi el mismo número de características que de muestras. Esto puede llevar a efectos indeseados por la "[maldición de la dimensionalidad](https://es.wikipedia.org/wiki/Maldici%C3%B3n_de_la_dimensi%C3%B3n)". Por este motivo, vamos a reducir el número de características con PCA:

In [ ]:
from sklearn.decomposition import PCA

Este modelo será entrenado para obtener el 95% de la varianza explicada:

In [ ]:
reductor = PCA(n_components=0.95).fit(features)
reduced_features = reductor.transform(features)
display(reduced_features.shape)

Como podemos ver, obtenemos casi la mitad de características con una representación más manejable.

## **4. K-Means**
---

El agrupamiento de textos es una técnica no supervisada que se utiliza para organizar y categorizar documentos de texto en grupos similares. El objetivo es agrupar los documentos de manera que los documentos dentro de un grupo sean similares entre sí y diferentes de los documentos en otros grupos.

<img src="https://drive.google.com/uc?export=view&id=15vVIbDqKp9qNQjB6G-r1g4VXZzswnuT8" width="80%">

El agrupamiento de textos es útil para tareas como la recuperación de información, la indexación de documentos y la organización automática de noticias o correos electrónicos. También puede ser utilizado para descubrir patrones y relaciones entre documentos, lo que puede ser útil en aplicaciones como la minería de datos y el análisis de sentimientos.

K-means es el algoritmo de agrupamiento más utilizado, en especial, cuando tenemos grandes conjuntos de datos. Este modelo busca dividir un conjunto de $N$ observaciones en $K$ grupos (donde $K$ es un número especificado previamente) de manera tal que las observaciones dentro de cada grupo son lo más similares posible entre sí.

<img src="https://drive.google.com/uc?export=view&id=1UNUKcthC49ZX7vPJfYy4b0yWe0cDCOJr" width="80%">

El algoritmo funciona iterativamente, asignando cada observación a su grupo más cercano (según la distancia Euclidiana) y luego recalculando el centroide (o punto medio) de cada grupo. Este proceso se repite hasta que las asignaciones de grupo y los centroides no cambian significativamente.

El algoritmo K-Means funciona de la siguiente manera:

1. Elegir el número de clusters (k) que se desea encontrar en los datos.
2. Seleccionar aleatoriamente k puntos de los datos como los centroides iniciales.
3. Asignar cada punto del conjunto de datos al cluster cuyo centroide esté más cerca (basado en la distancia Euclidiana).
4. Calcular los nuevos centroides de los clusters asignados.
5. Repetir los pasos 3 y 4 hasta que los centroides de los clusters no cambien o se alcance el número máximo de iteraciones.

En otras palabras, los puntos asignados a cada cluster se consideran como un grupo o cluster. K-Means es un algoritmo iterativo donde, en cada iteración, se asignan los puntos a los clusters más cercanos y se calculan los nuevos centroides. Los puntos se asignan a los nuevos clusters basados en los nuevos centroides. Este proceso se repite hasta que los centroides de los clusters no cambien o se alcance el número máximo de iteraciones.

Si se sigue este proceso, veremos cómo el algoritmo mueve los centroides hasta la convergencia, tal y como se muestra en la siguiente figura:

<img src="https://drive.google.com/uc?export=view&id=1Tu1p098KJYLJrvPJCruvQs3o5ol7vJTb" width="50%">

### **4.1. Implementación**
---

K-means se puede utilizar directamente desde `sklearn`. Vamos a importarlo:

In [ ]:
from sklearn.cluster import KMeans

Adicionalmente, debemos encontrar el número de clusters $K$ del modelo. Para ello, usaremos el coeficiente de silueta.

El **coeficiente de silueta (Silhouette Coefficient)** es una medida de cuán bien un punto dado está asignado a su grupo respectivo en un agrupamiento. Se utiliza para evaluar la calidad de un agrupamiento y para comparar diferentes algoritmos de clustering y diferentes configuraciones de parámetros. El coeficiente de silueta se calcula para cada punto individualmente y tiene un valor entre -1 y 1.

Para calcular el coeficiente de silueta de un punto, se utiliza la distancia promedio entre ese punto y los demás puntos en su mismo cluster (llamada "cohesión" o $a$) y la distancia promedio entre ese punto y los puntos en el cluster más cercano (llamada "separación" o $b$). El coeficiente de silueta se calcula como:

$$
( b - a ) / \text{max}(a, b).
$$

Un coeficiente de silueta cercano a 1 indica que el punto está muy bien asignado al cluster, mientras que un coeficiente cercano a -1 indica que el punto probablemente pertenezca a otro cluster. Valores cercanos a 0 indican que el punto está en el límite entre dos clusters y podría pertenecer a cualquiera de ellos.

El promedio de los coeficientes de silueta en todos los puntos es una medida global de la calidad del agrupamiento, donde un valor alto indica un agrupamiento de alta calidad y un valor bajo indica un agrupamiento de baja calidad.

Importamos esta métrica desde `sklearn`:

In [ ]:
from sklearn.metrics import silhouette_score

Definimos un rango del hiperparámetro $K$ que vamos a explorar:

In [ ]:
k_range = np.arange(2, 11, 1)
display(k_range)

Ahora, entrenamos varios modelos K-means para cada valor de $K$. Vamos a almacenar el coeficiente de silueta por cada uno, y también el modelo con mejor coeficiente de silueta:

In [ ]:
best_score = -1 # variable donde almacenamos el mejor modelo
metrics = [] # lista donde almacenamos las métricas
for k in k_range:
    model = KMeans(n_clusters=k, random_state=0, n_init=10).fit(reduced_features) # entrenamiento
    score = silhouette_score(
        reduced_features,
        model.predict(reduced_features)
        ) # evaluación
    metrics.append(score)
    if score > best_score: # validamos si la métrica mejora
        best_score = score # guardamos la mejor métrica
        best_model = model # guardamos el mejor modelo

Veamos una gráfica de las variaciones del coeficiente de silueta frente al número de clusters del modelo:

In [ ]:
fig, ax = plt.subplots()
ax.plot(k_range, metrics)
ax.set_xlabel("$K$")
ax.set_ylabel("Coeficiente de Silueta")
ax.set_xticks(k_range)
fig.show()

### **4.2. Interpretación**
---

Existen varias formas de interpretar los resultados de K-means para agrupamiento de textos. Primero, veamos la distribución de los clusters encontrados en el corpus:

In [ ]:
clusters = best_model.predict(reduced_features)
cats, counts = np.unique(clusters, return_counts=True)
fig, ax = plt.subplots()
ax.bar(cats, counts)
ax.set_xlabel("Cluster")
ax.set_ylabel("Conteo")
fig.show()

Como podemos ver, los clusters no se encuentran uniformemente distribuidos. Ahora, debemos tratar de interpretar qué es cada cluster. Para ello, vamos a agrupar los textos preprocesados del conjunto de datos original según el cluster asignado por el modelo de K-means. Para ello vamos a crear el siguiente `DataFrame`:

In [ ]:
predictions = pd.DataFrame({"text": df.norm_text, "cluster": clusters})
display(predictions.head())

Ahora, vamos a extraer todos los textos concatenados por cluster:

In [ ]:
grouped_texts = (
        predictions
        .groupby("cluster")
        .agg({"text": lambda series: " ".join(series)})
        .reset_index()
        )
display(grouped_texts)

Con esto podemos pasar a generar una visualización de tipo nube de palabras para los clusters encontrados, importamos `WordCloud`:

In [ ]:
from wordcloud import WordCloud

Ahora, generamos una gráfica para visualizar nubes de palabras de todos los textos dentro de un cluster específico.

In [ ]:
fig, axes = plt.subplots(
    1,
    best_model.n_clusters,
    figsize=(10 * best_model.n_clusters, 10),
    )
for cluster in range(best_model.n_clusters):
    ax = axes[cluster]
    ax.set_title(f"Cluster: {cluster}")
    text = grouped_texts.loc[grouped_texts.cluster == cluster, "text"].iloc[0]
    wc = WordCloud(
        background_color="#FFFFFF",
        width=500,
        height=500
        ).generate(text)
    ax.imshow(wc)
    ax.axis("off")
fig.show()

Los resultados pueden variar entre cada ejecución, no obstante, veamos un ejemplo de lo que se puede obtener:

<img src="https://drive.google.com/uc?export=view&id=1WzCVyYd6YYyxdTzCtAdDcK3BH4so7JzL" width="100%">

Podemos ver los siguientes patrones:

- Cluster 0, 1 y 5: probablemente hacen referencia a películas relacionadas a relaciones familiares o amistades.
- Cluster 2: parecen películas de deportes.
- Cluster 3: películas de adolescentes ambientadas en el colegio.
- Cluster 4: la temática puede estar relacionada a películas de drama.
- Cluster 6: probablemente son películas policiales o de crímenes.
- Cluster 7: películas sobre aliens o misiones espaciales.
- Cluster 8: películas de romance
- Cluster 9: películas con relaciones de hombre-mujer (esposos, padre e hija, entre otras).

Por último, podemos cruzar información categórica adicional a los clusters, como por ejemplo los géneros de las películas. Vamos a calcular los géneros totales por cluster:

In [ ]:
genres = pd.DataFrame({"genres": df.genres, "cluster": clusters})
grouped_genres = (
        genres
        .groupby("cluster")
        .agg({"genres": lambda genres: sum(map(list, genres.tolist()), [])})
        .reset_index()
        )

Ahora, podemos generar la distribución de géneros por cada cluster:

In [ ]:
fig, axes = plt.subplots(
    2, 5,
    figsize=(20, 20)
    )
for cluster in range(best_model.n_clusters):
    ax = axes[cluster // 5, cluster % 5]
    ax.set_title(f"Cluster: {cluster}")
    genres = grouped_genres.loc[grouped_genres.cluster == cluster, "genres"].iloc[0]
    unique, counts = np.unique(genres, return_counts=True)
    counts_df = (
            pd.DataFrame(data={"unique": unique, "counts": counts})
            .sort_values(by="counts", ascending=False)
            .head(5)
            )

    ax.barh(counts_df.unique, counts_df.counts)
    ax.set_xlabel("Conteos")
fig.tight_layout()
fig.show()

Nuevamente, los resultados pueden variar entre una ejecución y otra. Veamos una gráfica de la distribución de géneros para el ejemplo que veníamos mostrando:

<img src="https://drive.google.com/uc?export=view&id=17sosOpdIDsQwYD_llTxzGA_B2MXIk_Zt" width="100%">

Se pueden observar correspondencias con los patrones que identificamos en las nubes de palabras. Por ejemplo, el cluster 6 tiene una alta distribución de términos relacionados con el género de horror, suspenso y crimen, lo que se relaciona con temas de asesinos seriales, asesinatos, víctimas y similares. El cluster 7 tiene una alta distribución de términos relacionados con acción, aventura y ciencia ficción, lo que se relaciona con temáticas de aliens y espacio. El cluster 8 tiene una alta distribución de términos relacionados con romance, drama y comedia, lo que corresponde con palabras como amor.

## **5. Clustering Jerárquico**
---

Clustering jerárquico es un enfoque de agrupamiento no supervisado que se utiliza para organizar los datos en una estructura jerárquica de grupos o clusters. Existen dos tipos principales de clustering jerárquico:

- **Aglomerativo**: este enfoque comienza con cada punto como un cluster individual y luego agrupa gradualmente los puntos más similares en grupos más grandes hasta que se alcanza el nivel deseado de agrupamiento.
- **Divisivo**: este enfoque comienza con todos los puntos en un único grupo y luego divide gradualmente el grupo en subgrupos más pequeños hasta que se alcanza el nivel deseado de agrupamiento.

<img src="https://drive.google.com/uc?export=view&id=1ELvKAB4FvlMnveKPoqBPmq5e1jVhoNGU" width="80%">

El clustering jerárquico utiliza un enfoque de "todos contra todos" para comparar todos los puntos entre sí y determinar cuáles son los más similares. Luego, se utilizan diferentes criterios para unir o dividir los grupos. Uno de los criterios comúnmente utilizados es la distancia euclidiana entre los puntos.

Una de las ventajas del clustering jerárquico es que permite visualizar la estructura jerárquica de los datos mediante un dendrograma. Además, también permite seleccionar el nivel deseado de agrupamiento, lo que puede ser útil en algunas aplicaciones. Sin embargo, una desventaja es que puede ser computacionalmente costoso para conjuntos de datos muy grandes.

El clustering jerárquico se utiliza mucho en el procesamiento del lenguaje natural (NLP) debido a varias razones:

- Permite la exploración de los datos: El clustering jerárquico permite visualizar la estructura jerárquica de los datos mediante un dendrograma, lo que puede ser útil para entender la relación entre los documentos y descubrir patrones y temas emergentes.
- No requiere un número previamente especificado de clusters: A diferencia de otros algoritmos de clustering, como k-means, el clustering jerárquico no requiere que se especifique el número de clusters previamente. Esto es útil en el NLP, ya que a menudo es difícil saber cuántos grupos o temas hay en un conjunto de documentos.
- Permite la selección de nivel de agrupamiento: El clustering jerárquico permite seleccionar el nivel deseado de agrupamiento. Por ejemplo, se pueden crear clusters más generales para tener una vista panorámica de los datos o clusters más específicos para obtener una comprensión detallada de los temas.
- Es adecuado para datos no numéricos: El clustering jerárquico se utiliza a menudo con datos no numéricos, como texto, lo que lo hace adecuado para el NLP. El clustering jerárquico se puede aplicar a los documentos después de convertirlos en una representación numérica, utilizando técnicas como el modelo de lenguaje o la frecuencia de términos.

El enfoque aglomerativo es el tipo más común de clustering jerárquico. Comienza suponiendo que cada uno de los puntos es un cluster, los cuales se irán fusionando a lo largo de las iteraciones hasta obtener un número deseado de clusters. El método consiste en los siguientes pasos:

1. Calcular una matriz de similitud $\mathbf{S}$.
2. Fusionar los dos clusters más cercanos.
3. Repetir desde el paso 1 hasta obtener un único cluster o un número $K$ de clusters.

Como un cluster puede estar compuesto por varios puntos, existen diversas formas de obtener similitudes:

- `single`: la similitud entre dos clusters es la mínima similitud o distancia entre los puntos de cada cluster.

<img src="https://drive.google.com/uc?export=view&id=1VJbbWag36EJ_dEQ9Brwe6UPvNngY46-d" width="60%">

- `complete`: la similitud entre dos clusters es la máxima distancia entre los puntos de cada cluster.

<img src="https://drive.google.com/uc?export=view&id=1zQzghA8Sy75R-C0tniGHqKqTjL00OV1S" width="60%">

- `average`: la similitud entre dos clusters es el promedio entre la similitud de todas las combinaciones de puntos de cada cluster.

<img src="https://drive.google.com/uc?export=view&id=12Vqk_h6aDcrie2hiY-P-rctmsh56zqok" width="60%">

- `centroid`: la similitud entre dos clusters es la similitud entre los puntos promedio o centroides de cada cluster.

<img src="https://drive.google.com/uc?export=view&id=1IMYYFSGsEzL0V4VaUOQX4hibM01eOvUF" width="60%">

- `ward`: la similitud entre dos clusters es la suma de cuadrados media de todas las combinaciones de puntos de cada cluster.

### **5.1. Implementación**
---

Veamos la aplicación de agglomerative clustering, comenzamos importando las funciones necesarias:

In [ ]:
from scipy.cluster.hierarchy import linkage

Para aplicar el clustering jerárquico debemos obtener una matriz conocida como _linkage matrix_, la cual tiene tamaño $(N-1) \times 4$. Cada fila $i$ contiene el resultado de cada iteración, y en las columnas se encuentra la siguiente información: $L_{i,0}$ y $L_{i,1}$ contienen los índices de los clusters que se unirán, $L_{i,2}$ contiene la similitud entre los dos clusters y $L_{i,3}$ contiene el número de puntos totales que hay en el cluster formado.

Para aplicar clustering jerárquico se debe construir una matriz de similitud entre cada documento. En este caso se utiliza la distancia Euclidiana.

La función `linkage` tiene los siguientes parámetros:

- `y`: matriz de características de los datos.
- `method`: método de unión aglomerativo.
- `metric`: medida de similitud usada para comparar muestras.

In [ ]:
linkage_matrix = linkage(reduced_features, method="ward", metric="euclidean")
display(linkage_matrix)

### **5.2. Interpretación**
---

Un dendrograma es un tipo de diagrama utilizado para representar la estructura jerárquica de los datos en un clustering jerárquico. Se ve como una especie de árbol con ramas y hojas, donde las ramas representan los clusters y las hojas representan los puntos individuales.

<img src="https://drive.google.com/uc?export=view&id=1gIn3v1dhQ843TMhvP93zR_HHiuPVKDyj" width="80%">

Cada nivel del dendrograma representa un nivel de agrupamiento diferente, con los clusters más generales en la parte superior y los clusters más específicos en la parte inferior. Los puntos se agrupan en clusters más pequeños a medida que se desciende por el dendrograma.

Las ramas del dendrograma se unen mediante un punto de unión, que representa el punto en el que se combinaron dos clusters para crear un cluster más grande. La distancia entre los puntos de unión también puede ser utilizada para medir la similitud entre los clusters.

El dendrograma permite visualizar la estructura jerárquica de los datos y seleccionar el nivel deseado de agrupamiento. También es útil para detectar patrones y relaciones entre los puntos, y para comparar diferentes algoritmos de clustering y diferentes configuraciones de parámetros.

Esta visualización se genera usando la función `dendrogram`

In [ ]:
from scipy.cluster.hierarchy import dendrogram

Vamos a generar la visualización para el resultado de clustering jerárquico y mostrando el título de cada película:

In [ ]:
titles = df.title.tolist()

Construimos el dendrograma (árbol jerárquico):

In [ ]:
R = dendrogram(
    linkage_matrix, orientation="left",
    labels=titles, truncate_mode='lastp',
    p=100, no_plot=True,
    )

Definimos una función para etiquetar cada una de las ramas del árbol.

In [ ]:
def llf(x):
    # Asignamos cada uno de los títulos de las películas a cada una de las ramas
    temp = {R["leaves"][i]: titles[i] for i in range(len(R["leaves"]))}
    return "{}".format(temp[x])

Visualizamos el dendrograma:

In [ ]:
fig = plt.figure(figsize=(10, 40))
ax = dendrogram(linkage_matrix, truncate_mode='lastp', orientation="left", p=100,
              leaf_label_func=llf, leaf_font_size=10.)
fig.tight_layout()
fig.show()

Esta visualización nos permite interpretar qué películas se parecen más entre sí, de acuerdo a su sinopsis, con lo que podríamos llegar a identificar grupos de películas similares.

Por último, también es posible usar clustering jerárquico como cualquier otro modelo de clustering de `sklearn` de la siguiente forma:

In [ ]:
from sklearn.cluster import AgglomerativeClustering

Esta clase tiene los siguientes parámetros:

- `n_clusters`: permite especificar cuántos clusters deseamos obtener.
- `metric`: medida de similitud a usar para comparar muestras.
- `linkage`: permite especificar el tipo de unión que se usará.

Veamos un ejemplo:

In [ ]:
hc = (
        AgglomerativeClustering(n_clusters=3, linkage="ward")
        .fit(reduced_features)
        )

Veamos las etiquetas asignadas:

In [ ]:
labels = hc.labels_
display(labels[:10])

Por último, veamos cuántas películas quedaron agrupadas en cada uno de los 3 clusters:

In [ ]:
pd.value_counts(labels)

## **Recursos Adicionales**
---

Los siguientes enlaces corresponden a sitios donde encontrará información muy útil para profundizar en los temas vistos en este taller guiado:

- [Hierarchical clustering - scipy](https://docs.scipy.org/doc/scipy/reference/cluster.hierarchy.html).
- [Clustering - sklearn](https://scikit-learn.org/stable/modules/clustering.html).

## **Créditos**

* **Profesor:** [Felipe Restrepo Calle](https://dis.unal.edu.co/~ferestrepoca/)
* **Asistentes docentes:**
    - [Juan Sebastián Lara Ramírez](https://www.linkedin.com/in/juan-sebastian-lara-ramirez-43570a214/).
* **Diseño de imágenes:**
    - [Rosa Alejandra Superlano Esquibel](mailto:rsuperlano@unal.edu.co).
* **Coordinador de virtualización:**
    - [Edder Hernández Forero](https://www.linkedin.com/in/edder-hernandez-forero-28aa8b207/).

**Universidad Nacional de Colombia** - *Facultad de Ingeniería*